In [1]:
import sys
import tensorflow as tf
import keras
import gym
# import keras_rl2 # Keras-RL2 is often imported as part of the DQN agent creation later

print(f"Python version: {sys.version}")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"Gym version: {gym.__version__}")

print("\nAll required libraries imported successfully!")

2025-06-09 09:14:49.195544: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-09 09:14:49.206261: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-09 09:14:49.225214: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-09 09:14:49.251731: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749460489.285946    1637 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749460489.29

Python version: 3.9.17 (main, Jun 13 2023, 16:07:34) 
[GCC 8.3.0]
TensorFlow version: 2.19.0
Keras version: 3.10.0
Gym version: 0.26.2

All required libraries imported successfully!


In [4]:
import gym
import numpy as np

# 1. 初始化 Acrobot 环境
# 使用 legacy_wrapped_env=True 以确保与旧版 Gym 兼容性，
# 并避免一些未来版本中可能存在的API变化。
try:
    env = gym.make("Acrobot-v1", legacy_wrapped_env=True, render_mode="human") # human 模式用于可视化
    print("Acrobot-v1 environment created successfully with human render_mode.")
except Exception as e:
    env = gym.make("Acrobot-v1", render_mode="rgb_array") # 如果 human 模式报错，则使用 rgb_array 模式
    print(f"Could not create Acrobot-v1 with human render_mode, falling back to rgb_array. Error: {e}")
    print("Acrobot-v1 environment created successfully with rgb_array render_mode.")


# 2. 检索和理解状态空间
observation_space = env.observation_space
print(f"\nObservation Space (State Space): {observation_space}")
print(f"  Shape: {observation_space.shape}")
print(f"  Low values: {observation_space.low}")
print(f"  High values: {observation_space.high}")

# Acrobot 的状态空间解释：
# 这是一个 Box(6,) 类型，意味着它是一个包含6个浮点值的数组。
# 这6个值代表：
# - cos(theta1): 第一个连杆的角度余弦值
# - sin(theta1): 第一个连杆的角度正弦值
# - cos(theta2): 第二个连杆的角度余弦值
# - sin(theta2): 第二个连杆的角度正弦值
# - theta1_dot: 第一个连杆的角速度
# - theta2_dot: 第二个连杆的角速度
# 这些值是连续的，范围通常在 -1 到 1 之间（对于cos/sin），角速度有其最大最小值。

# 3. 检索和理解动作空间
action_space = env.action_space
print(f"\nAction Space: {action_space}")
print(f"  Number of actions: {action_space.n}")

# Acrobot 的动作空间解释：
# 这是一个 Discrete(3) 类型，意味着代理有3个离散的动作可供选择：
# - 0: 施加负扭矩（-1）到第二个连杆的关节
# - 1: 不施加扭矩（0）
# - 2: 施加正扭矩（+1）到第二个连杆的关节
# 目标是通过这些扭矩使下连杆摆动到垂直位置。

# 重置环境以获取初始状态
initial_state, info = env.reset()
print(f"\nInitial state after reset: {initial_state}")

# 关闭环境（最佳实践，释放资源）
env.close()

Could not create Acrobot-v1 with human render_mode, falling back to rgb_array. Error: __init__() got an unexpected keyword argument 'legacy_wrapped_env'
Acrobot-v1 environment created successfully with rgb_array render_mode.

Observation Space (State Space): Box([ -1.        -1.        -1.        -1.       -12.566371 -28.274334], [ 1.        1.        1.        1.       12.566371 28.274334], (6,), float32)
  Shape: (6,)
  Low values: [ -1.        -1.        -1.        -1.       -12.566371 -28.274334]
  High values: [ 1.        1.        1.        1.       12.566371 28.274334]

Action Space: Discrete(3)
  Number of actions: 3

Initial state after reset: [ 0.9997121   0.02399328  0.9997853  -0.02072063 -0.03496813 -0.07802942]


In [1]:
import gym
import time # 用于暂停，方便观察
import numpy as np

# 重新创建环境，用于随机动作的观察
env = gym.make("Acrobot-v1", render_mode="human") # 确保是 human 模式以便可视化

num_random_episodes = 3 # 运行3个随机剧集进行观察
max_steps_per_episode = 200 # 每个剧集的最大步数

print(f"Running {num_random_episodes} random episodes...")

for episode in range(num_random_episodes):
    state, info = env.reset() # 重置环境
    done = False
    truncated = False # 对于 Gym 0.21+，除了 done 还有 truncated
    total_reward = 0
    step_count = 0

    print(f"\n--- Episode {episode + 1} ---")
    while not done and not truncated and step_count < max_steps_per_episode:
        action = env.action_space.sample() # 从动作空间中随机选择一个动作
        
        # 执行动作，获取新状态、奖励、是否结束、是否截断、额外信息
        next_state, reward, done, truncated, info = env.step(action)
        
        total_reward += reward
        step_count += 1
        
        # 实时渲染环境，以便观察
        env.render()
        
        # （可选）暂停片刻以便观察动画
        # time.sleep(0.01)

        state = next_state # 更新当前状态

        # 可以打印一些信息来观察
        # if step_count % 50 == 0:
        #     print(f"  Step {step_count}: Action={action}, Reward={reward:.2f}, Total Reward={total_reward:.2f}")

    print(f"Episode {episode + 1} finished after {step_count} steps. Total Reward: {total_reward:.2f}")

# 关闭环境
env.close()
print("\nFinished running random episodes.")

Running 3 random episodes...

--- Episode 1 ---
Episode 1 finished after 200 steps. Total Reward: -200.00

--- Episode 2 ---
Episode 2 finished after 200 steps. Total Reward: -200.00

--- Episode 3 ---
Episode 3 finished after 200 steps. Total Reward: -200.00

Finished running random episodes.


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 根据 Acrobot 环境定义输入和输出维度
state_size = 6  # 从 env.observation_space.shape[0] 获取
action_size = 3 # 从 env.action_space.n 获取

# 定义模型构建函数
def build_q_network(input_shape, output_size):
    model = keras.Sequential([
        keras.Input(shape=input_shape), # 输入层，与状态空间形状匹配
        layers.Dense(64, activation="relu", name="hidden_layer_1"), # 第一个隐藏层，64个神经元，ReLU激活
        layers.Dense(64, activation="relu", name="hidden_layer_2"), # 第二个隐藏层，64个神经元，ReLU激活
        layers.Dense(output_size, activation="linear", name="output_layer") # 输出层，与动作空间大小匹配，线性激活（Q值）
    ])
    return model

# 实例化模型
q_network = build_q_network(input_shape=(state_size,), output_size=action_size)

print("Neural Network Architecture Designed.")

2025-06-09 12:54:07.740235: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-09 12:54:07.896084: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-09 12:54:08.605411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-09 12:54:08.605633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-09 12:54:08.747491: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Neural Network Architecture Designed.


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 确保定义了状态空间和动作空间的大小
state_size = 6
action_size = 3

# 构建 Q 网络模型
model = keras.Sequential([
    keras.Input(shape=(state_size,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(action_size, activation='linear')
])

print("Keras Neural Network Model Implemented.")

Keras Neural Network Model Implemented.


In [4]:
# 打印模型摘要
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                448       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 4803 (18.76 KB)
Trainable params: 4803 (18.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
